Concept of measurement: observations that *quantitatively* reduce the uncertainty.

## Object of measurement
Security is vague. decompose it to observable quantities.

### How the risk matrix can be inconsistent

![risk-matrix.png](images/risk-matrix.png)

It can be difficult to make a naively constructed risk matrix consistent, when we put numbers in, i.e., "Medium" cells could have loss larger than "Critical" ones.

### Measure the "immesurable"

#### rule-of-five
Given five observations, the possibility that the median of the population is inside the range (min, max) is as high as 93.75%. 

In [10]:
p_inside = (1 - 0.5**5 * 2)  # all the five larger (or lower) than the median of the population

In [11]:
p_inside

0.9375